In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [2]:
with open('./Assignment/chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]
#print(stopwords)  

In [3]:
import jieba
news = pd.read_csv('./Assignment/sqlResult.csv',encoding = 'gb18030')
print(news.shape)
print(news.head(5))

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [4]:
print(news.isnull().sum())

id             0
author     10215
source         2
content     2557
feature        0
title         34
url         2467
dtype: int64


In [5]:
print(news[news.content.isna()].head())
news = news.dropna(subset=['content'])
print(news.shape)

         id author     source content  \
100   89517    NaN  中国证券报?中证网     NaN   
103   89514    NaN  中国证券报?中证网     NaN   
997   88620    NaN        央广网     NaN   
1273  88344    NaN        央广网     NaN   
1282  88335    NaN        央广网     NaN   

                                                feature  \
100   {"type":"公司","site":"中证网","commentNum":"0","jo...   
103   {"type":"公司","site":"中证网","commentNum":"0","jo...   
997   {"type":"时事要闻","site":"参考消息","commentNum":"0",...   
1273  {"type":"IT业界","site":"参考消息","commentNum":"0",...   
1282  {"type":"IT业界","site":"参考消息","commentNum":"0",...   

                                 title  \
100       天和防务股东未来6个月内计划减持不超过480万股公司股份   
103                    晶盛机电调整限制性股票回购价格   
997              [主播不在家]第二季：主播陈亮体验垃圾清运   
1273                LKK洛可可：想象力经济时代或已到来   
1282  CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台   

                                                    url  
100   http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...  
103   http://www.cs.com.cn/ss

In [6]:
def split_text(text):
    text = text.replace(' ','')
    text = text.replace('\n','')
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [7]:
print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\邢开源\AppData\Local\Temp\jieba.cache


此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。



Loading model cost 0.606 seconds.
Prefix dict has been built succesfully.


 当然 MIUI9 确切 信息 等待 官方消息 


In [8]:
import pickle
import os
if not os.path.exists('corpus.pkl'):
    corpus = list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    with open('corpus.pkl','wb') as file :
        pickle.dump(corpus,file)
else:
    with open('corpus.pkl','rb')as file:
        corpus = pickle.load(file)

In [8]:
countvectorizer  = CountVectorizer(encoding = 'gb18030',min_df = 0.015)
tfidftransformer = TfidfTransformer()

countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)

In [9]:
label = list(map(lambda source : 1 if '新华' in str(source) else 0,news.source))

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3) 

In [11]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
copy_news_index = compare_news_index[(compare_news_index['prediction']==1)&(compare_news_index['labels']==0)].index 
print('可能为copy的新闻',len(copy_news_index))

可能为copy的新闻 2825


In [13]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

In [14]:
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)
if not os.path.exists('label.pkl'):
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels,file)
else:
    with open('label.pkl','rb')as file:
        k_labels = pickle.load(file)

In [23]:
id_calss = {index:class_ for index,class_ in enumerate(k_labels)}
if not os.path.exists('id_class.pkl'):
    with open('id_class.pkl','wb') as file :
        pickle.dump(id_calss,file)
else:
    with open('id_class.pkl','rb') as file :
        id_class = pickle.load(file)
    
xinhua_news_index = compare_news_index[(compare_news_index['labels']==1)].index
from collections import  defaultdict
class_id = defaultdict(set)
for index,class_ in id_calss.items():
    if index in xinhua_news_index.tolist():
        class_id[class_].add(index)
if not os.path.exists('class_id.pkl'):
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id,file)
else:
    with open('class_id.pkl','rb')as file:
        class_id = pickle.load(file)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similar_text(cpindex,top =10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(),key = lambda x:x[1][0],reverse = True)[:top]

In [27]:
import editdistance

In [31]:
cpindex = 0

similar_list = get_similar_text(cpindex)
print('怀疑抄袭：\n',news.iloc[cpindex].content)

similar2 = similar_list[0][0]
print('相似原文：\n',news.iloc[similar2].content)

print('编辑距离：',editdistance.eval(corpus[cpindex],corpus[similar2]))

怀疑抄袭：
 此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

相似原文：
 　　新华社北京4月10日电（记者孔祥鑫）记者10日从北京市住建委获悉，北京市对“商改住”进行限购以来，严格查处违规发布商办类项目用于居住的中介信息。近日，北京市住建委会同市工商局、市网信办对安居客房地产信息网站违规发布商办项目房源信息进行执法检查。目前，安居客已经下架全部商办项目房源信息，涉事房地产中介公司已被注销备案。\n　　经执法检查发现，房地产中介机构和个人以“独立经纪人”名义在安居客网站发布相关信息，存在违规代理商办类房屋销售或者虚假宣传商办类房屋居住用途的情况。安居客网站作为网络信息发布平台，对此类信息未核查，违反了北京市相关规定。\n　　对于违规发布信息的北京中天置地房地产经纪有限公司第二分公司、北京恒洋我家房地产经纪有限公司第十四分公司、北京我淘我家房地产经纪有限公司回龙观分公司等三家房地产中介公司，执法部门已经责令关停并注销经纪机构备案。同时，北京市网信办责令安居客下架全部涉事房源信息，并要求安居客对今后在网站上发布的房源信息进行充分核实。\n　　此外，在检查中还发现，有部分所谓“独立经纪人”在安居客等网站上发布房源信息。《房地产经纪管理办法》明确规定，房地产经纪业务应当由房地产经纪机构统一承接，房地产经纪人员不得以个人名义承接房地产经纪业务。这类“独立经纪人”及其所在的经纪机构将受到严查。\n　　下一步，北京市住建委将会同工商、网络管理部门开展专项执法行动，重点查处房地产经纪机构在网站上发布虚假房源信息、以“独立经纪人”个人名义在网站发布房源信息等行为，并加大执法检查力度，规范房地产类网站平台的经营活动。（完）\n\n
编辑距离： 761
